In [2]:
import argparse
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras


In [3]:
model = "MLP"
labels = 1
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [4]:
csv_path = 'jena_climate_2009_2016.csv'
df = pd.read_csv(csv_path)

In [5]:
column_indices = [2, 5]
columns = df.columns[column_indices]
data = df[columns].values.astype(np.float32)

In [6]:
n = len(data)
train_data = data[0:int(n*0.7)]
val_data = data[int(n*0.7):int(n*0.9)]
test_data = data[int(n*0.9):]

In [7]:
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)

In [8]:
input_width = 6
LABEL_OPTIONS = labels

In [9]:
class WindowGenerator:
    def __init__(self, input_width, label_options, mean, std):
        self.input_width = input_width
        self.label_options = label_options
        self.mean = tf.reshape(tf.convert_to_tensor(mean), [1, 1, 2])
        self.std = tf.reshape(tf.convert_to_tensor(std), [1, 1, 2])

    def split_window(self, features):
        inputs = features[:, :-1, :]

        if self.label_options < 2:
            labels = features[:, -1, self.label_options]
            labels = tf.expand_dims(labels, -1)
            num_labels = 1
        else:
            labels = features[:, -1, :]
            num_labels = 2

        inputs.set_shape([None, self.input_width, 2])
        labels.set_shape([None, num_labels])

        return inputs, labels

    def normalize(self, features):
        features = (features - self.mean) / (self.std + 1.e-6)

        return features

    def preprocess(self, features):
        inputs, labels = self.split_window(features)
        inputs = self.normalize(inputs)

        return inputs, labels

    def make_dataset(self, data, train):
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
                data=data,
                targets=None,
                sequence_length=input_width+1,
                sequence_stride=1,
                batch_size=32)
        ds = ds.map(self.preprocess)
        ds = ds.cache()
        if train is True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True)

        return ds

In [10]:
generator = WindowGenerator(input_width, LABEL_OPTIONS, mean, std)
train_ds = generator.make_dataset(train_data, True)
val_ds = generator.make_dataset(val_data, False)
test_ds = generator.make_dataset(test_data, False)

In [12]:
model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1)
])

In [13]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MAE,
              metrics=['MSE'])

In [16]:
model.fit(train_ds, batch_size=32, epochs=1)
#model.fit(train_ds, batch_size=32, epochs=20)

9200/9200 [==============================] - 15s 2ms/step - loss: 0.9661 - MSE: 1.8571


In [18]:
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print('\n MSE:', test_acc)


 MSE: 2.850781202316284
